
Creating Task Manager with User Authentication 

Problem Statement

In today’s world, individuals often need to keep track of various tasks in a structured way. You are tasked with building a Task Manager that allows users to manage their tasks.

The system should include user authentication, meaning each user has to log in with a username and password. Once logged in, users can create, view, update, and delete their tasks. Each user’s tasks should be stored separately, and only the authenticated user can access their tasks.

Problem Objective: 

1.    Design and implement a user authentication system (login and registration)
2.    Create a task management system that allows users to:
a.    Add, view, mark as completed, and delete tasks
3.    Use file handling to store user credentials and tasks persistently
4.    Create an interactive menu-driven interface to manage tasks

 

Steps to Perform: 

1.  User Authentication:
•    Registration: 
     Create a function to prompt the user to enter a username and password
     Ensure that the username is unique, and hash the password for security before storing it in a file

•    Login: 
     Create a function to prompt the user for their username and password, validate the credentials by comparing them with the stored data, and grant access to the task manager upon successful login


2.  Add a Task:
•    Create a function that prompts the user for a task description. Assign a unique task ID and set the status to Pending
•    Store the task in a file, and confirm that the task was added

 

3.  View Tasks:
•    Create a function to retrieve and display all tasks for the logged-in user
•    Each task should show the task ID, description, and status (Pending or Completed)

 

4.  Mark a Task as Completed:
•    Create a function that allows the user to select a task by its ID and update its status to Completed

 

5.  Delete a Task:
•    Create a function that allows the user to select a task by its ID and delete it from their task list

 

6.  Create an Interactive Menu:
•    Build a menu that allows users to choose between:
      Add a Task
      View Tasks
      Mark a Task as Completed
     Delete a Task
     Logout
For each option, call the corresponding function, and loop back to the menu until the user logs out.


In [4]:
import hashlib
import csv
import os
import uuid

# --- Constants ---
USERS_FILE = "users.txt"
TASKS_DIR = "user_tasks"

# --- Utility Functions ---

def ensure_tasks_dir_exists():
    """Ensures the directory for storing user tasks exists."""
    if not os.path.exists(TASKS_DIR):
        os.makedirs(TASKS_DIR)

def get_user_tasks_file_path(username):
    """Returns the file path for a user's tasks."""
    # Changed file extension to .csv
    return os.path.join(TASKS_DIR, f"{username}_tasks.csv")

def hash_password(password):
    """Hashes a password for secure storage."""
    return hashlib.sha256(password.encode()).hexdigest()

def load_user_credentials():
    """Loads user credentials from the users file."""
    if not os.path.exists(USERS_FILE):
        return {}
    
    credentials = {}
    with open(USERS_FILE, 'r') as f:
        for line in f:
            if ':' in line:
                username, hashed_password = line.strip().split(':', 1)
                credentials[username] = hashed_password
    return credentials

def save_user_credentials(credentials):
    """Saves user credentials to the users file."""
    with open(USERS_FILE, 'w') as f:
        for username, hashed_password in credentials.items():
            f.write(f"{username}:{hashed_password}\n")

def load_tasks(username):
    """Loads tasks for a specific user from their CSV file."""
    tasks_file = get_user_tasks_file_path(username)
    if not os.path.exists(tasks_file):
        return []
    
    tasks = []
    try:
        with open(tasks_file, mode='r', newline='', encoding='utf-8') as f:
            # Use DictReader to read tasks as dictionaries
            reader = csv.DictReader(f)
            for row in reader:
                tasks.append(row)
    except Exception as e:
        print(f"An error occurred while reading the tasks file: {e}")
    return tasks

def save_tasks(username, tasks):
    """Saves tasks for a specific user to their CSV file."""
    tasks_file = get_user_tasks_file_path(username)
    fieldnames = ['id', 'description', 'status']
    
    try:
        with open(tasks_file, mode='w', newline='', encoding='utf-8') as f:
            # Use DictWriter to write list of dictionaries to CSV
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            if tasks:
                writer.writerows(tasks)
    except Exception as e:
        print(f"An error occurred while saving the tasks file: {e}")


# --- User Authentication ---

def register_user():
    """Handles new user registration."""
    print("\n--- Register a New User ---")
    credentials = load_user_credentials()
    
    while True:
        username = input("Enter a new username: ")
        if not username:
            print("Username cannot be empty.")
            continue
        if username in credentials:
            print("Username already exists. Please choose another one.")
        else:
            break
            
    while True:
        password = input("Enter a password: ")
        if not password:
            print("Password cannot be empty.")
        else:
            break

    hashed_password = hash_password(password)
    credentials[username] = hashed_password
    save_user_credentials(credentials)
    print(f"\nUser '{username}' registered successfully!")

def login_user():
    """Handles user login and returns the username on success."""
    print("\n--- User Login ---")
    credentials = load_user_credentials()
    
    while True:
        username = input("Enter your username: ")
        if not username:
            print("Username cannot be empty.")
            continue
        password = input("Enter your password: ")
        if not password:
            print("Password cannot be empty.")
            continue

        if username not in credentials:
            print("Invalid username or password.")
            return None

        hashed_input_password = hash_password(password)
        if credentials[username] == hashed_input_password:
            print(f"\nWelcome, {username}! Login successful.")
            return username
        else:
            print("Invalid username or password.")
            return None

# --- Task Management ---

def add_task(username):
    """Adds a new task for the logged-in user."""
    print("\n--- Add a New Task ---")
    task_description = input("Enter the task description: ")
    if not task_description:
        print("Task description cannot be empty.")
        return
    
    # Ensure description with commas doesn't break CSV format
    if ',' in task_description:
        print("For simplicity, please avoid using commas in the task description.")
        task_description = task_description.replace(',', '')


    tasks = load_tasks(username)
    new_task = {
        "id": str(uuid.uuid4()),
        "description": task_description,
        "status": "Pending"
    }
    tasks.append(new_task)
    save_tasks(username, tasks)
    print("\nTask added successfully!")

def view_tasks(username):
    """Displays all tasks for the logged-in user."""
    print("\n--- Your Tasks ---")
    tasks = load_tasks(username)
    
    if not tasks:
        print("You have no tasks.")
        return

    for task in tasks:
        print(f"  ID: {task['id']}")
        print(f"  Description: {task['description']}")
        print(f"  Status: {task['status']}")
        print("-" * 20)

def mark_task_completed(username):
    """Marks a specific task as completed."""
    print("\n--- Mark a Task as Completed ---")
    view_tasks(username)
    tasks = load_tasks(username)

    if not tasks:
        return

    task_id = input("Enter the ID of the task to mark as completed: ")
    task_found = False
    for task in tasks:
        if task['id'] == task_id:
            task['status'] = 'Completed'
            task_found = True
            break
            
    if task_found:
        save_tasks(username, tasks)
        print("\nTask marked as completed!")
    else:
        print("\nTask ID not found.")

def delete_task(username):
    """Deletes a specific task."""
    print("\n--- Delete a Task ---")
    view_tasks(username)
    tasks = load_tasks(username)

    if not tasks:
        return

    task_id = input("Enter the ID of the task to delete: ")
    
    initial_task_count = len(tasks)
    tasks = [task for task in tasks if task['id'] != task_id]
    
    if len(tasks) < initial_task_count:
        save_tasks(username, tasks)
        print("\nTask deleted successfully!")
    else:
        print("\nTask ID not found.")

# --- Interactive Menus ---

def task_menu(username):
    """Displays the main task management menu."""
    while True:
        print("\n--- Task Manager Menu ---")
        print("1. Add a Task")
        print("2. View Tasks")
        print("3. Mark a Task as Completed")
        print("4. Delete a Task")
        print("5. Logout")
        
        choice = input("Enter your choice (1-5): ")
        
        if choice == '1':
            add_task(username)
        elif choice == '2':
            view_tasks(username)
        elif choice == '3':
            mark_task_completed(username)
        elif choice == '4':
            delete_task(username)
        elif choice == '5':
            print("\nLogging out. Goodbye!")
            break
        else:
            print("\nInvalid choice. Please enter a number between 1 and 5.")

def main():
    """The main function to run the application."""
    ensure_tasks_dir_exists()
    
    while True:
        print("\n--- Welcome to the Task Manager ---")
        print("1. Login")
        print("2. Register")
        print("3. Exit")
        
        choice = input("Enter your choice (1-3): ")
        
        if choice == '1':
            logged_in_username = login_user()
            if logged_in_username:
                task_menu(logged_in_username)
        elif choice == '2':
            register_user()
        elif choice == '3':
            print("\nExiting the Task Manager. Have a great day!")
            break
        else:
            print("\nInvalid choice. Please enter 1, 2, or 3.")

if __name__ == "__main__":
    main()



--- Welcome to the Task Manager ---
1. Login
2. Register
3. Exit

--- Register a New User ---
Username already exists. Please choose another one.
Username cannot be empty.
Username cannot be empty.
Username cannot be empty.
Username cannot be empty.
Username cannot be empty.

User '2' registered successfully!

--- Welcome to the Task Manager ---
1. Login
2. Register
3. Exit

--- Register a New User ---
Username already exists. Please choose another one.
Username already exists. Please choose another one.

User '3' registered successfully!

--- Welcome to the Task Manager ---
1. Login
2. Register
3. Exit

Exiting the Task Manager. Have a great day!
